In [1]:
        import pandas as pd
        import numpy as np
        import math
        import matplotlib.pyplot as plt
        
        import warnings
        warnings.filterwarnings('ignore')
        
        pd.set_option('display.max_columns',None)
        pd.set_option('display.max_row',None)
        pd.set_option('display.expand_frame_repr',False)
        pd.set_option('display.max_colwidth',None)

In [2]:
deliveries = pd.read_csv('ipl_2024_deliveries.csv')
matches = pd.read_csv('ipl_2024_matches.csv')

In [3]:
df = deliveries

In [4]:
def ByInnings(df,current_innings):
    
    df = df[df.innings == current_innings]
    df.reset_index(inplace = True,drop = True)
    
    df['isDot'] = df['runs_of_bat'].apply(lambda x: 1 if x == 0 else 0)
    df['isOne'] = df['runs_of_bat'].apply(lambda x: 1 if x == 1 else 0)
    df['isTwo'] = df['runs_of_bat'].apply(lambda x: 1 if x == 2 else 0)
    df['isThree'] = df['runs_of_bat'].apply(lambda x: 1 if x == 3 else 0)
    df['isFour'] = df['runs_of_bat'].apply(lambda x: 1 if x == 4 else 0)
    df['isSix'] = df['runs_of_bat'].apply(lambda x: 1 if x == 6 else 0)
    
    runs = pd.DataFrame(df.groupby(['batsman'])['runs_of_bat'].sum()).reset_index().rename(columns={'runs_of_bat':'runs'})
    innings = pd.DataFrame(df.groupby(['batsman'])['match_id'].apply(lambda x: len(list(np.unique(x))))).reset_index().rename(columns={'match_id':'innings'})
    balls = pd.DataFrame(df.groupby(['batsman'])['match_id'].count()).reset_index().rename(columns={'match_id':'balls'})
    dismissals = pd.DataFrame(df.groupby(['batsman'])['player_dismissed'].count()).reset_index().rename(columns={'player_dismissed':'dismissals'})
    sixes = pd.DataFrame(df.groupby(['batsman'])['isSix'].sum()).reset_index().rename(columns = {'isSix':'sixes'})
    fours = pd.DataFrame(df.groupby(['batsman'])['isFour'].sum()).reset_index().rename(columns = {'isFour':'fours'})
    
    df = pd.merge(innings,runs,on = 'batsman').merge( balls, on = 'batsman').merge( dismissals, on = 'batsman').merge( fours, on = 'batsman').merge( sixes, on = 'batsman')
    df['RPI'] =df.apply(lambda x: (x['runs']/x['innings']),axis = 1)
    return df


In [5]:
df.head()

,match_id,season,match_no,date,venue,batting_team,bowling_team,innings,over,striker,bowler,runs_of_bat,extras,wide,legbyes,byes,noballs,wicket_type,player_dismissed,fielder
0,202401,2024,1,"Mar 22, 2024","MA Chidambaram Stadium, Chennai",RCB,CSK,1,0.1,Kohli,Chahar,0,1,1,0,0,0,NaN,NaN,NaN
1,202401,2024,1,"Mar 22, 2024","MA Chidambaram Stadium, Chennai",RCB,CSK,1,0.1,Kohli,Chahar,1,0,0,0,0,0,NaN,NaN,NaN
2,202401,2024,1,"Mar 22, 2024","MA Chidambaram Stadium, Chennai",RCB,CSK,1,0.2,du Plessis,Chahar,0,0,0,0,0,0,NaN,NaN,NaN
3,202401,2024,1,"Mar 22, 2024","MA Chidambaram Stadium, Chennai",RCB,CSK,1,0.3,du Plessis,Chahar,0,0,0,0,0,0,NaN,NaN,NaN
4,202401,2024,1,"Mar 22, 2024","MA Chidambaram Stadium, Chennai",RCB,CSK,1,0.4,du Plessis,Chahar,0,0,0,0,0,0,NaN,NaN,NaN


In [6]:
def balls_per_dismissal(balls, dismissals):
    if dismissals > 0:
        return balls/dismissals
    else:
        return balls/1 
    
def balls_per_boundary(balls, boundaries):
    if boundaries > 0:
        return balls/boundaries
    else:
        return balls/1 

In [63]:
def ByCustom(df, current_venue, current_phase, current_opposition):
    
    df = df[df.venue == current_venue]
    df = df[df.phase == current_phase]
    df = df[df.bowling_team == current_opposition]
   
  
    df.reset_index(inplace = True,drop = True)
    
    df['isDot'] = df['runs_of_bat'].apply(lambda x: 1 if x == 0 else 0)
    df['isOne'] = df['runs_of_bat'].apply(lambda x: 1 if x == 1 else 0)
    df['isTwo'] = df['runs_of_bat'].apply(lambda x: 1 if x == 2 else 0)
    df['isThree'] = df['runs_of_bat'].apply(lambda x: 1 if x == 3 else 0)
    df['isFour'] = df['runs_of_bat'].apply(lambda x: 1 if x == 4 else 0)
    df['isSix'] = df['runs_of_bat'].apply(lambda x: 1 if x == 6 else 0)
    
    runs = pd.DataFrame(df.groupby(['batsman'])['runs_of_bat'].sum()).reset_index().rename(columns={'runs_of_bat':'runs'})
    innings = pd.DataFrame(df.groupby(['batsman'])['match_id'].apply(lambda x: len(list(np.unique(x))))).reset_index().rename(columns={'match_id':'innings'})
    balls = pd.DataFrame(df.groupby(['batsman'])['match_id'].count()).reset_index().rename(columns={'match_id':'balls'})
    dismissals = pd.DataFrame(df.groupby(['batsman'])['player_dismissed'].count()).reset_index().rename(columns={'player_dismissed':'dismissals'})
    sixes = pd.DataFrame(df.groupby(['batsman'])['isSix'].sum()).reset_index().rename(columns = {'isSix':'sixes'})
    fours = pd.DataFrame(df.groupby(['batsman'])['isFour'].sum()).reset_index().rename(columns = {'isFour':'fours'})

    dots = pd.DataFrame(df.groupby(['batsman'])['isDot'].sum()).reset_index().rename(columns = {'isDot':'dots'})

    
    df = pd.merge(innings,runs,on = 'batsman').merge( balls, on = 'batsman').merge( dismissals, on = 'batsman').merge( fours, on = 'batsman').merge( sixes, on = 'batsman').merge( dots, on = 'batsman')
    df['RPI'] = df.apply(lambda x: x['runs']/x['innings'], axis = 1)
    df['SR'] = df.apply(lambda x: 100*(x['runs']/x['balls']),axis = 1)
    df['BPD'] = df.apply(lambda x: balls_per_dismissal(x['balls'], x['dismissals']), axis = 1)
    df['BPB'] = df.apply(lambda x: balls_per_boundary(x['balls'], (x['fours'] + x['sixes'])), axis = 1)

    df['dot_percentage'] = df.apply(lambda x: x['dots']/x['balls'], axis = 1)

    return df

In [65]:
df=deliveries.copy()

mdf=matches.copy()

comb=pd.merge(df,mdf,on = 'match_id',how ='left')

comb = comb.rename(columns={'striker':'batsman'})

comb = comb.rename(columns={'venue_x':'venue'})

In [75]:
def ByCustom(df, current_venue, current_phase, current_opposition):
    # Filtering by venue & phase
    df = df[(df['venue'] == current_venue) & (df['phase'] == current_phase)].copy()
    
    # Keeping both teams' batting data
    df = df[(df['bowling_team'] == current_opposition) | (df['batting_team'] == current_opposition)]

    # Continue with existing calculations...
    df.reset_index(drop=True, inplace=True)

    # Derived Metrics (Same as Before)
    df['isDot'] = (df['runs_of_bat'] == 0).astype(int)
    df['isFour'] = (df['runs_of_bat'] == 4).astype(int)
    df['isSix'] = (df['runs_of_bat'] == 6).astype(int)

    stats = df.groupby('batsman').agg(
        runs=('runs_of_bat', 'sum'),
        innings=('match_id', 'nunique'),
        balls=('match_id', 'count'),
        dismissals=('player_dismissed', lambda x: x.notna().sum()),
        fours=('isFour', 'sum'),
        sixes=('isSix', 'sum'),
        dots=('isDot', 'sum')
    ).reset_index()

    stats['RPI'] = stats['runs'] / stats['innings']  
    stats['SR'] = (stats['runs'] / stats['balls']) * 100  
    stats['BPD'] = stats.apply(lambda x: balls_per_dismissal(x['balls'], x['dismissals']), axis=1)  
    stats['BPB'] = stats.apply(lambda x: balls_per_boundary(x['balls'], x['fours'] + x['sixes']), axis=1)  
    stats['dot_percentage'] = stats['dots'] / stats['balls']  

    return stats


In [77]:
def get_phase(over_no):
    if over_no < 6:
        return'PowerPlay'
        
    elif over_no < 15:
        return'Middle Over'
    else:
        return'Death Over'

In [79]:
comb['phase'] = comb['over'].apply(lambda x: get_phase(x))

In [97]:
df = ByCustom(comb, 'Eden Gardens, Kolkata', 'PowerPlay', 'KKR')
df.sort_values(by='runs', ascending=False).head()


,batsman,runs,innings,balls,dismissals,fours,sixes,dots,RPI,SR,BPD,BPB,dot_percentage
15,Philip Salt,213,7,119,4,24,14,53,30.428571,178.991597,29.750000,3.131579,0.445378
12,Narine,94,7,68,3,14,3,33,13.428571,138.235294,22.666667,4.000000,0.485294
16,Prabhsimran,54,1,21,0,4,5,6,54.000000,257.142857,21.000000,2.333333,0.285714
26,Venkatesh Iyer,46,3,22,1,7,2,8,15.333333,209.090909,22.000000,2.444444,0.363636
27,Will Jacks,40,1,19,0,3,4,9,40.000000,210.526316,19.000000,2.714286,0.473684


In [85]:
df = ByCustom(comb ,'Eden Gardens, Kolkata','Middle Over','KKR')

In [87]:
df.head(15)

,batsman,runs,innings,balls,dismissals,fours,sixes,dots,RPI,SR,BPD,BPB,dot_percentage
0,Abdul Samad,2,1,5,0,0,0,3,2.0,40.000000,5.0,5.000000,0.600000
1,Abhishek Sharma,5,1,5,1,1,0,3,5.0,100.000000,5.0,5.000000,0.600000
2,Abishek Porel,0,1,3,1,0,0,3,0.0,0.000000,3.0,3.000000,1.000000
3,Angkrish Raghuvanshi,15,1,11,1,2,0,2,15.0,136.363636,11.0,5.500000,0.181818
4,Ashwin,8,1,12,1,1,0,7,8.0,66.666667,12.0,12.000000,0.583333
5,Axar,15,1,21,1,2,0,13,15.0,71.428571,21.0,10.500000,0.619048
6,Ayush Badoni,24,1,21,1,1,1,5,24.0,114.285714,21.0,10.500000,0.238095
7,Bairstow,61,1,26,0,3,6,6,61.0,234.615385,26.0,2.888889,0.230769
8,Buttler,38,1,28,0,5,0,9,38.0,135.714286,28.0,5.600000,0.321429
9,Dhruv Jurel,2,1,4,1,0,0,2,2.0,50.000000,4.0,4.000000,0.500000


In [89]:
df = ByCustom(comb ,'Eden Gardens, Kolkata','Death Over','KKR')

In [91]:
df.head(15)

,batsman,runs,innings,balls,dismissals,fours,sixes,dots,RPI,SR,BPD,BPB,dot_percentage
0,Abdul Samad,13,1,6,1,1,1,1,13.0,216.666667,6.0,3.000000,0.166667
1,Anshul Kamboj,2,1,2,0,0,0,0,2.0,100.000000,2.0,2.000000,0.000000
2,Arshad Khan,5,1,4,1,1,0,2,5.0,125.000000,4.0,4.000000,0.500000
3,Bairstow,11,1,6,0,0,1,1,11.0,183.333333,6.0,6.000000,0.166667
4,Boult,0,1,1,0,0,0,1,0.0,0.000000,1.0,1.000000,1.000000
5,Buttler,49,1,23,1,2,5,8,49.0,213.043478,23.0,3.285714,0.347826
6,Chawla,1,1,2,0,0,0,1,1.0,50.000000,2.0,2.000000,0.500000
7,Cummins,0,1,1,0,0,0,1,0.0,0.000000,1.0,1.000000,1.000000
8,Ferguson,1,1,1,1,0,0,0,1.0,100.000000,1.0,1.000000,0.000000
9,Karn Sharma,20,1,8,1,0,3,3,20.0,250.000000,8.0,2.666667,0.375000


In [17]:
comb['venue'].unique()

array(['MA Chidambaram Stadium, Chennai',
       'Maharaja Yadavindra Singh International Cricket Stadium, Mullanpur, Chandigarh',
       'Eden Gardens, Kolkata', 'Sawai Mansingh Stadium, Jaipur',
       'Narendra Modi Stadium, Ahmedabad',
       'M.Chinnaswamy Stadium, Bengaluru',
       'Rajiv Gandhi International Stadium, Hyderabad',
       'Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium, Lucknow',
       'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium, Visakhapatnam',
       'Wankhede Stadium, Mumbai', 'Arun Jaitley Stadium, Delhi',
       'Himachal Pradesh Cricket Association Stadium, Dharamsala',
       'Barsapara Cricket Stadium, Guwahati'], dtype=object)

In [18]:
wt_sr, wt_rpi, wt_bpd, wt_dot_percentage = 0.13, 0.27, 0.16, 0.45

In [19]:
#step1: square of all values
df['calc_SR'] = df['SR'].apply(lambda x: x*x) 
df['calc_RPI'] = df['RPI'].apply(lambda x: x*x) 
df['calc_BPD'] = df['BPD'].apply(lambda x: x*x) 
df['calc_dot_percentage'] = df['dot_percentage'].apply(lambda x: x*x)

sq_sr, sq_rpi, sq_bpd, sq_dot_percentage = np.sqrt(df[['calc_SR','calc_RPI', 'calc_BPD', 'calc_dot_percentage']].sum(axis = 0))

df['calc_SR'] = df['calc_SR'].apply(lambda x: x/sq_sr) 
df['calc_RPI'] = df['calc_RPI'].apply(lambda x: x/sq_rpi) 
df['calc_BPD'] = df['calc_BPD'].apply(lambda x: x/sq_bpd) 
df['calc_dot_percentage'] = df['calc_dot_percentage'].apply(lambda x: x/sq_dot_percentage)

df['calc_SR'] = df['calc_SR'].apply(lambda x: x*wt_sr) 
df['calc_RPI'] = df['calc_RPI'].apply(lambda x: x*wt_rpi) 
df['calc_BPD'] = df['calc_BPD'].apply(lambda x: x*wt_bpd) 
df['calc_dot_percentage'] = df['calc_dot_percentage'].apply(lambda x: x*wt_dot_percentage)

best_sr, worst_sr = max(df['calc_SR']), min(df['calc_SR'])
best_rpi, worst_rpi = max(df['calc_RPI']), min(df['calc_RPI'])
best_bpd, worst_bpd = max(df['calc_BPD']), min(df['calc_BPD'])
best_dot_percentage, worst_dot_percentage = min(df['calc_dot_percentage']), max(df['calc_dot_percentage'])

In [20]:
df['dev_best_SR'] = df['calc_SR'].apply(lambda x: (x-best_sr)*(x-best_sr)) 
df['dev_best_RPI'] = df['calc_RPI'].apply(lambda x: (x-best_rpi)*(x-best_rpi)) 
df['dev_best_BPD'] = df['calc_BPD'].apply(lambda x: (x-best_bpd)*(x-best_bpd)) 
df['dev_best_dot_percentage'] = df['calc_dot_percentage'].apply(lambda x: (x-best_dot_percentage)*(x-best_dot_percentage))

df['dev_best_sqrt'] = df.apply(lambda x: x['dev_best_SR'] + x['dev_best_RPI'] + x['dev_best_BPD'] + x['dev_best_dot_percentage'], axis = 1) 

df['dev_worst_SR'] = df['calc_SR'].apply(lambda x: (x-worst_sr)*(x-worst_sr)) 
df['dev_worst_RPI'] = df['calc_RPI'].apply(lambda x: (x-worst_rpi)*(x-worst_rpi)) 
df['dev_worst_BPD'] = df['calc_BPD'].apply(lambda x: (x-worst_bpd)*(x-worst_bpd)) 
df['dev_worst_dot_percentage'] = df['calc_dot_percentage'].apply(lambda x: (x-worst_dot_percentage)*(x-worst_dot_percentage))

df['dev_worst_sqrt'] = df.apply(lambda x: x['dev_worst_SR'] + x['dev_worst_RPI'] + x['dev_worst_BPD'] + x['dev_worst_dot_percentage'], axis = 1) 

In [21]:
df['score'] = df.apply(lambda x: x['dev_worst_sqrt']/(x['dev_worst_sqrt'] + x['dev_best_sqrt']), axis = 1)

In [22]:
df[[ 'batsman','innings', 'runs', 'balls', 'dismissals', 'dot_percentage', 'score']].sort_values(['runs'], ascending = False).reset_index(drop = True).head(50)

,batsman,innings,runs,balls,dismissals,dot_percentage,score
0,Shashank Singh,1,49,17,0,0.235294,0.955453
1,Buttler,1,49,23,1,0.347826,0.531437
2,Klaasen,1,45,18,1,0.277778,0.753562
3,Pooran,1,34,21,1,0.428571,0.151707
4,Kuldeep Yadav,1,25,22,0,0.545455,0.038677
5,Powell,1,24,8,1,0.250000,0.880348
6,Karn Sharma,1,20,8,1,0.375000,0.591419
7,Karthik,1,20,15,1,0.400000,0.043736
8,Shahbaz Ahmed,1,16,5,1,0.400000,0.889187
9,Abdul Samad,1,13,6,1,0.166667,0.330580
